<a href="https://www.kaggle.com/code/priyam8210/sign-2b?scriptVersionId=246468347" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import warnings 
warnings.filterwarnings('ignore')
%config InlineBackend.figure_format ='retina'
sns.set(style='ticks')

plt.rc('figure', figsize=(6, 3.7), dpi=100) # figure size
plt.rc('axes', labelpad=20, facecolor="#ffffff", # properties of axes
       linewidth=0.4, grid=True, labelsize=10) 
plt.rc('xtick.major', width=0.2) # major ticks where the labels are shown
plt.rc('ytick.major', width=0.2) # minor ticks where the labels are not shown
plt.rc('grid', color='#EEEEEE', linewidth=0.25)
plt.rc('font', family='Arial', weight='400', size=10)
plt.rc('text', color='#282828')
plt.rc('xtick', labelsize=10)
plt.rc('ytick', labelsize=10)
plt.rc('savefig', pad_inches=0.3, dpi=300)

In [2]:
TRAIN_FILE = '/kaggle/input/asl-signs/train.csv'

In [3]:
img = pd.read_parquet('/kaggle/input/asl-signs/train_landmark_files/16069/100015657.parquet')

In [4]:
img.shape

(57015, 7)

In [5]:
import torch.nn as nn

In [6]:
class datagen(nn.Module):
    def __init__(self):
        super(datagen, self).__init__()
        pass
    def forward(self, x):
    #n = len(x['frame'].value_counts())
    #x = torch.tensor(x.iloc[:, 4:].to_numpy())
    #x = x.contiguous().view(n, 543, -1)
        face_x = x[:,:468,:].contiguous().view(-1, 468*3)
        lefth_x = x[:,468:489,:].contiguous().view(-1, 21*3)
        pose_x = x[:,489:522,:].contiguous().view(-1, 33*3)
        righth_x = x[:,522:,:].contiguous().view(-1, 21*3)

        lefth_x = lefth_x[~torch.any(torch.isnan(lefth_x), dim=1),:]
        righth_x = righth_x[~torch.any(torch.isnan(righth_x), dim=1),:]

        x1m = torch.mean(face_x, 0)
        x2m = torch.mean(lefth_x, 0)
        x3m = torch.mean(pose_x, 0)
        x4m = torch.mean(righth_x, 0)

        x1s = torch.std(face_x, 0)
        x2s = torch.std(lefth_x, 0)
        x3s = torch.std(pose_x, 0)
        x4s = torch.std(righth_x, 0)

        xfeat = torch.cat([x1m,x2m,x3m,x4m, x1s,x2s,x3s,x4s], axis=0)
        xfeat = torch.where(torch.isnan(xfeat), torch.tensor(0.0, dtype=torch.float32), xfeat)

        return xfeat
    


In [7]:
d_gen = datagen()

In [8]:
df = pd.read_csv('/kaggle/input/asl-signs/train.csv')
df

,path,participant_id,sequence_id,sign
0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow
1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait
2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud
3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird
4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie
...,...,...,...,...
94472,train_landmark_files/53618/999786174.parquet,53618,999786174,white
94473,train_landmark_files/26734/999799849.parquet,26734,999799849,have
94474,train_landmark_files/25571/999833418.parquet,25571,999833418,flower
94475,train_landmark_files/29302/999895257.parquet,29302,999895257,room


In [9]:
df.values

array([['train_landmark_files/26734/1000035562.parquet', 26734,
        1000035562, 'blow'],
       ['train_landmark_files/28656/1000106739.parquet', 28656,
        1000106739, 'wait'],
       ['train_landmark_files/16069/100015657.parquet', 16069, 100015657,
        'cloud'],
       ...,
       ['train_landmark_files/25571/999833418.parquet', 25571, 999833418,
        'flower'],
       ['train_landmark_files/29302/999895257.parquet', 29302, 999895257,
        'room'],
       ['train_landmark_files/36257/999962374.parquet', 36257, 999962374,
        'happy']], dtype=object)

In [10]:
import json

with open('/kaggle/input/asl-signs/sign_to_prediction_index_map.json', 'r') as f:
    data = json.load(f)

In [11]:
from tqdm import tqdm
import os
import torch

In [12]:
ROWS_PER_FRAME = 543
def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / 543)
    data = data.values.reshape(n_frames, 543, len(data_columns))
    return data.astype(np.float32)

In [13]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import random_split
from torch.utils.data import DataLoader


In [14]:
tr = pd.DataFrame(np.load('/kaggle/input/isrl-basic-data-mean/feature_data.npy'))
out = pd.DataFrame(np.load('/kaggle/input/isrl-basic-data-mean/feature_labels.npy'))

In [15]:
class data(Dataset):
    def __init__(self, df, label):
        self.df = df
        self.label = label
        
    def __getitem__(self, index):
        inp = torch.tensor(self.df.values[index]).float()    
        target = torch.tensor(self.label.values[index]).long()
        return inp, target
    
    def __len__(self):
        return len(self.df)

In [16]:
tr_d = data(tr, out)

In [17]:
tr_d[0]

(tensor([ 0.5277,  0.3780, -0.0373,  ...,  0.1577,  0.0730,  0.0426]),
 tensor([25]))

In [18]:
tr_ds, val_ds = random_split(tr_d, [0.80, 0.20])
tr_dl = DataLoader(tr_ds, batch_size=256, shuffle=True, num_workers=8, pin_memory=True)
val_dl = DataLoader(val_ds, batch_size=512, num_workers=8, pin_memory=True)

In [19]:
def get_default_device():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [20]:
device = get_default_device()
device

device(type='cuda')

In [21]:
tr_dl = DeviceDataLoader(tr_dl, device)
val_dl = DeviceDataLoader(val_dl, device)

In [22]:
import torch.nn as nn

In [23]:
import torch.nn.functional as F


In [24]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [25]:
class lstm_model(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = nn.Linear(3258, 1024)
        self.linear2 = nn.Linear(1024, 512)
        self.linear3= nn.Linear(512, 250 )
        self.dropout = nn.Dropout(0.2)
        self.los = nn.CrossEntropyLoss()
        
    def forward(self, x):
        out= self.linear1(x)
        out = self.dropout(out)
        out = self.linear2(out)
        out = self.dropout(out)
        out = self.linear3(out)
        return out

    def training_step(self, batch):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.los(outputs, targets.squeeze())
        return loss

    def validation_step(self, batch):
        inputs, targets = batch
        outputs = self(inputs)
        loss = self.los(outputs, targets.squeeze())
        #acc = accuracy(out, idx)           
        return {'val_loss': loss.detach()}#, 'val_acc': acc.detach()}

    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        #batch_accs = [x['val_acc'] for x in outputs]
       # epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item()}#, 'val_acc': epoch_acc.item()}

    def epoch_end(self, epoch, result):
       print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss']))

In [26]:
model = to_device(lstm_model(), device)
model

lstm_model(
  (linear1): Linear(in_features=3258, out_features=1024, bias=True)
  (linear2): Linear(in_features=1024, out_features=512, bias=True)
  (linear3): Linear(in_features=512, out_features=250, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (los): CrossEntropyLoss()
)

In [27]:
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit_one_cycle(epochs, max_lr, model, train_loader, val_loader, 
                  weight_decay=0, grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    # Set up cutom optimizer with weight decay
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # Set up one-cycle learning rate scheduler
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epochs, 
                                                steps_per_epoch=len(train_loader))
    
    for epoch in range(epochs):
        # Training Phase 
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # Gradient clipping
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizer.step()
            optimizer.zero_grad()
            
            # Record & update learning rate
            lrs.append(get_lr(optimizer))
            sched.step()
        
        # Validation phase
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
    return history

In [28]:
history = [evaluate(model, val_dl)]
history

[{'val_loss': 5.52331018447876}]

In [29]:
epochs = 150
max_lr = 0.006
grad_clip = 0.1
weight_decay = 1e-6
opt_func = torch.optim.Adam

In [30]:
%%time
history += fit_one_cycle(epochs, max_lr, model, tr_dl, val_dl, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=opt_func)

Epoch [0], last_lr: 0.00025, train_loss: 5.4580, val_loss: 5.2369
Epoch [1], last_lr: 0.00027, train_loss: 5.0153, val_loss: 4.8952
Epoch [2], last_lr: 0.00030, train_loss: 4.6587, val_loss: 4.4687
Epoch [3], last_lr: 0.00035, train_loss: 4.3126, val_loss: 4.2053
Epoch [4], last_lr: 0.00041, train_loss: 4.0911, val_loss: 3.9956
Epoch [5], last_lr: 0.00049, train_loss: 3.9350, val_loss: 3.9460
Epoch [6], last_lr: 0.00058, train_loss: 3.8198, val_loss: 3.7496
Epoch [7], last_lr: 0.00068, train_loss: 3.7635, val_loss: 3.8041
Epoch [8], last_lr: 0.00079, train_loss: 3.6960, val_loss: 3.7658
Epoch [9], last_lr: 0.00091, train_loss: 3.6930, val_loss: 3.8586
Epoch [10], last_lr: 0.00105, train_loss: 3.6270, val_loss: 3.8211
Epoch [11], last_lr: 0.00119, train_loss: 3.6193, val_loss: 3.8906
Epoch [12], last_lr: 0.00135, train_loss: 3.5988, val_loss: 3.7102
Epoch [13], last_lr: 0.00151, train_loss: 3.6072, val_loss: 3.4265
Epoch [14], last_lr: 0.00168, train_loss: 3.6057, val_loss: 3.4560
Epoch

In [31]:
def plot_accuracies(history):
    accuracies = [x['val_loss'] for x in history]
    plt.plot(accuracies, '-x')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.title('Accuracy vs. No. of epochs');

In [33]:
!pip install onnx-tf
!pip install tflite-runtime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 6.0 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.9 MB/s eta 0:00:00a 0:00:01


In [34]:
sample_input = torch.rand((50, 543, 3))
onnx_feat_gen_path = 'feature_gen.onnx'

d_gen.eval()

torch.onnx.export(
    d_gen,                  # PyTorch Model
    sample_input,                    # Input tensor
    onnx_feat_gen_path,        # Output file (eg. 'output_model.onnx')
    opset_version=12,       # Operator support version
    input_names=['input'],   # Input tensor name (arbitary)
    output_names=['output'], # Output tensor name (arbitary)
    dynamic_axes={
        'input' : {0: 'input'}
    }
)

In [35]:
sample_input = torch.rand((1, 3258)).cuda()
onnx_model_path = 'asl_model.onnx'

model.eval()

torch.onnx.export(
    model,                  # PyTorch Model
    sample_input,                    # Input tensor
    onnx_model_path,        # Output file (eg. 'output_model.onnx')
    opset_version=12,       # Operator support version
    input_names=['input'],   # Input tensor name (arbitary)
    output_names=['output'], # Output tensor name (arbitary)
    dynamic_axes={
        'input' : {0: 'input'}
    }
)

In [36]:
import onnx
from onnx_tf.backend import prepare


tf_feat_gen_path = '/kaggle/working/tf_feat_gen'
onnx_feat_gen = onnx.load(onnx_feat_gen_path)
tf_rep = prepare(onnx_feat_gen)
tf_rep.export_graph(tf_feat_gen_path)


tf_model_path = '/kaggle/working/tf_model'
onnx_model = onnx.load(onnx_model_path)
tf_rep = prepare(onnx_model)
tf_rep.export_graph(tf_model_path)

In [37]:
import tensorflow as tf

class ASLInferModel(tf.Module):
    def __init__(self):
        super(ASLInferModel, self).__init__()
        self.feature_gen = tf.saved_model.load(tf_feat_gen_path)
        self.model = tf.saved_model.load(tf_model_path)
        self.feature_gen.trainable = False
        self.model.trainable = False
    
    @tf.function(input_signature=[
      tf.TensorSpec(shape=[None, 543, 3], dtype=tf.float32, name='inputs')
    ])
    def call(self, input):
        output_tensors = {}
        features = self.feature_gen(**{'input': input})['output']
        output_tensors['outputs'] = self.model(**{'input': tf.expand_dims(features, 0)})['output'][0,:]
        return output_tensors
    
    
mytfmodel = ASLInferModel()
tf.saved_model.save(mytfmodel, '/kaggle/working/tf_infer_model', signatures={'serving_default': mytfmodel.call})

In [38]:
tf_infer_model_path = '/kaggle/working/tf_infer_model'
converter = tf.lite.TFLiteConverter.from_saved_model(tf_infer_model_path)
tflite_model = converter.convert()

tflite_model_path = 'model.tflite'

# Save the model
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

In [39]:
ROWS_PER_FRAME = 543  # number of landmarks per frame
pq_path = "/kaggle/input/asl-signs/train_landmark_files/18796/1001373962.parquet"


import tflite_runtime.interpreter as tflite
interpreter = tflite.Interpreter(tflite_model_path)
interpreter.allocate_tensors()

found_signatures = list(interpreter.get_signature_list().keys())

# if REQUIRED_SIGNATURE not in found_signatures:
#     raise KernelEvalException('Required input signature not found.')

prediction_fn = interpreter.get_signature_runner("serving_default")
output = prediction_fn(inputs=load_relevant_data_subset(pq_path))
sign = np.argmax(output["outputs"])

print(sign, output["outputs"].shape)

5 (250,)


In [40]:
!zip submission.zip $tflite_model_path

  adding: model.tflite (deflated 7%)
